# Joins

## Prerrequisites

Install Spark and Java in VM

In [12]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [13]:
ls -l # check the .tgz is there

total 391016
drwxr-xr-x 1 root root      4096 Jan 11 17:02 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [14]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [15]:
!pip install -q findspark

In [16]:
!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [17]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [18]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Joins") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [19]:
spark

In [20]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [21]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/guitars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/guitarPlayers.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/bands.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/employees.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/salaries.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/deptmanagers.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/titles.csv -P /dataset
!ls /dataset

bands.json	  employees.csv       guitars.json  titles.csv
deptmanagers.csv  guitarPlayers.json  salaries.csv


Read JSONs

In [22]:
guitarsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitars.json")

guitaristsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitarPlayers.json")

bandsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/bands.json")

In [23]:
guitarsDF.show(3)
guitaristsDF.show(3)
bandsDF.show(3)


+--------------------+---+------+------------+
|          guitarType| id|  make|       model|
+--------------------+---+------+------------+
|Electric double-n...|  0|Gibson|    EDS-1275|
|            Electric|  5|Fender|Stratocaster|
|            Electric|  1|Gibson|          SG|
+--------------------+---+------+------------+
only showing top 3 rows

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+
only showing top 3 rows

+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
+-----------+---+------------+----+
only showing top 3 rows



## Examples

Inner Join =  everything from BOTH DFs for which there is a row in the right DF satisfying the condition


In [44]:
joinCondition = guitaristsDF.band == bandsDF.id
guitaristsBandsDF = guitaristsDF.join(bandsDF, joinCondition, "inner")
guitaristsBandsDF.show(3)

+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+



In [25]:
# things to bear in mind
# guitaristsBandsDF.select("id", "band").show(3) # will crash because there are two columns named "id"

In [26]:
# you can rename the column
bandsDFMod = bandsDF.withColumnRenamed("id", "bandId") \
    .withColumnRenamed("name", "bandName")
guitaristsBandsDF2 = guitaristsDF.join(bandsDFMod,
    guitaristsDF.band == bandsDFMod.bandId, "inner").orderBy("name")
guitaristsBandsDF2.show(3)

# there are other options but it is the best practice if it is possible

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|     3|   Metallica|1981|
+----+-------+---+------------+-----------+------+------------+----+



Left Outer = everything in the inner join + all the rows in the LEFT DF, with nulls in where the data is missing

In [27]:
joinCondition2 = guitaristsDF.band == bandsDFMod.bandId
guitaristsDF.join(bandsDFMod, joinCondition2, "left_outer").orderBy("name").show(3)
# see nulls on Eric Calpton

+----+-------+---+------------+--------+------+------------+----+
|band|guitars| id|        name|hometown|bandId|    bandName|year|
+----+-------+---+------------+--------+------+------------+----+
|   1|    [1]|  1| Angus Young|  Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|    NULL|  NULL|        NULL|NULL|
|   0|    [0]|  0|  Jimmy Page|  London|     0|Led Zeppelin|1968|
+----+-------+---+------------+--------+------+------------+----+
only showing top 3 rows



  Right Outer = everything in the inner join + all the rows in the RIGHT DF, with nulls in where the data is missing


In [28]:
guitaristsDF.join(bandsDFMod, joinCondition2, "right_outer").orderBy("name").show(3)
# see nulls for The Beatles

+----+-------+----+-----------+---------+------+------------+----+
|band|guitars|  id|       name| hometown|bandId|    bandName|year|
+----+-------+----+-----------+---------+------+------------+----+
|NULL|   NULL|NULL|       NULL|Liverpool|     4| The Beatles|1960|
|   1|    [1]|   1|Angus Young|   Sydney|     1|       AC/DC|1973|
|   0|    [0]|   0| Jimmy Page|   London|     0|Led Zeppelin|1968|
+----+-------+----+-----------+---------+------+------------+----+
only showing top 3 rows



 Outer Join = everything in the inner join + all the rows in BOTH DFs, with nulls in where the data is missing


In [29]:
guitaristsDF.join(bandsDFMod, joinCondition2, "outer").orderBy("name").show(3)
# see nulls for The Beatles and Eric Clapton

+----+-------+----+------------+---------+------+-----------+----+
|band|guitars|  id|        name| hometown|bandId|   bandName|year|
+----+-------+----+------------+---------+------+-----------+----+
|NULL|   NULL|NULL|        NULL|Liverpool|     4|The Beatles|1960|
|   1|    [1]|   1| Angus Young|   Sydney|     1|      AC/DC|1973|
|   2| [1, 5]|   2|Eric Clapton|     NULL|  NULL|       NULL|NULL|
+----+-------+----+------------+---------+------+-----------+----+
only showing top 3 rows



  Semi-joins = everything in the LEFT DF for which there is a row in the right DF satisfying the condition


In [30]:
guitaristsDF.join(bandsDFMod, joinCondition2, "left_semi").orderBy("name").show(3)


+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   1|    [1]|  1| Angus Young|
|   0|    [0]|  0|  Jimmy Page|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



Anti-joins = everything in the LEFT DF for which there is NO row in the right DF satisfying the condition


In [31]:
guitaristsDF.join(bandsDFMod, joinCondition2, "anti").orderBy("name").show(3)

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



Cross-join = takes every instance in the LEFT DF and create a new one for every intance in the right DF (not very common)

In [32]:
guitaristsDF.crossJoin(bandsDFMod).show(20)

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   0|    [0]|  0|  Jimmy Page|     Sydney|     1|       AC/DC|1973|
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|     Sydney|     1|       AC/DC|1973|
|   3|    [3]|  3|Kirk Hammett|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     London|     0|Led Zeppelin|1968|
|   2| [1, 5]|  2|Eric Clapton|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|     London|     0|Led Zeppelin|1968|
|   0|    [0]|  0|  Jimmy Page|Los Angeles|     3|   Metallica|1981|
|   1|    [1]|  1| Angus Young|Los Angeles|     3|   Metallica|1981|
|   2| [1, 5]|  2|Eric Clapton|Los Angeles|     3|   Metallica|1981|
|   3|    [3]|  3|Kirk Hammett|Los

*Union Join is just an Union (note it only works if both DFs have the same schema)

## Exercises
1. Read employees.csv, deptmanagers.csv, salaries.csv and titles.csv to a DFs.
2. Show all employees and their max salary (there can be different salaries registered for the same employee)
3. Show all employees who were never managers. Check it by getting all the managers and checking that they are not in the table
4. Find the job titles of the best paid 10 employees in the company (note that there can be different titles registered for the same employee)

Exercise 1

In [100]:
employeesDF = spark.read.option("header", "true").csv("/dataset/employees.csv")
employeesDF.show(3)

salariesDF = spark.read.option("header", "true").csv("/dataset/salaries.csv")
salariesDF.show(3)

deptManagersDF = spark.read.option("header", "true").csv("/dataset/deptmanagers.csv")
deptManagersDF.show(3)


titlesDF = spark.read.option("header", "true").csv("/dataset/titles.csv")
titlesDF.show(3)


# Schema for employeesDF
employeesDF.printSchema()

# Schema for salariesDF
salariesDF.printSchema()

# Schema for managersDF
deptManagersDF.printSchema()

# Schema for titlesDF
titlesDF.printSchema()



+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|  Demeyer|     M|1994-02-17|
+------+----------+----------+---------+------+----------+
only showing top 3 rows

+------+------+----------+----------+
|emp_no|salary| from_date|   to_date|
+------+------+----------+----------+
| 10010| 72488|1996-11-24|1997-11-24|
| 10010| 74347|1997-11-24|1998-11-24|
| 10010| 75405|1998-11-24|1999-11-24|
+------+------+----------+----------+
only showing top 3 rows

+-------+------+----------+----------+
|dept_no|emp_no| from_date|   to_date|
+-------+------+----------+----------+
|   d004|110420|1996-08-30|9999-01-01|
|   d006|110800|1991-09-12|1994-06-28|
|   d008|111400|1985-01-01|1991-04-08|
+-------+------+----------+----------

Exercise 2

In [48]:
#Show all employees and their max salary (there can be different salaries registered for the same employee)

joinCondition = employeesDF.emp_no == salariesDF.emp_no
employeesDFsalariesDF = employeesDF.join(salariesDF, joinCondition, "inner")
employeesDFsalariesDF.show(3)

# Group by employeesDF's emp_no and calculate the maximum salary for each employee
max_salariesDF = employeesDFsalariesDF.groupBy(employeesDF.emp_no).agg(max("salary").alias("max_salary"))

# Show the result
max_salariesDF.show()



+------+----------+----------+---------+------+----------+------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|salary| from_date|   to_date|
+------+----------+----------+---------+------+----------+------+------+----------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 72488|1996-11-24|1997-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 74347|1997-11-24|1998-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 75405|1998-11-24|1999-11-24|
+------+----------+----------+---------+------+----------+------+------+----------+----------+
only showing top 3 rows

+------+----------+
|emp_no|max_salary|
+------+----------+
|100000|     64961|
|100010|     71957|
|100020|     70286|
|100030|     66691|
|100040|     78188|
|100050|     73868|
|100060|     72296|
|100070|     75646|
|100080|     59704|
|100090|     70825|
| 10010|     80324|
|100100|     69230|
|100110|  

Exercise 3

In [83]:
joinCondition = employeesDF.emp_no == deptManagersDF.emp_no
employeesDFManagersDF = employeesDF.join(deptManagersDF, joinCondition, "left_anti")

employeesDFManagersDF.filter(col("emp_no")== 110420).show(3)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
+------+----------+----------+---------+------+----------+



Exercise 4

In [104]:

#Show all employees and their max salary (there can be different salaries registered for the same employee)
joinCondition = employeesDF.emp_no == salariesDF.emp_no
employeesDFsalariesDF = employeesDF.join(salariesDF, joinCondition, "inner")
employeesDFsalariesDF.show(3)

# Group by first_name and last_name, calculate the maximum salary for each group
max_salary_per_employee = employeesDFsalariesDF.groupBy("first_name", "last_name") \
    .agg(F.max("salary").alias("max_salary"))

# Order the results in descending order of maximum salary
sorted_max_salaries = max_salary_per_employee.orderBy(F.desc("max_salary"))

# Show the top 10 employees with their maximum salary
sorted_max_salaries.show(10)


!ls /dataset

+------+----------+----------+---------+------+----------+------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|salary| from_date|   to_date|
+------+----------+----------+---------+------+----------+------+------+----------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 72488|1996-11-24|1997-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 74347|1997-11-24|1998-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 10010| 75405|1998-11-24|1999-11-24|
+------+----------+----------+---------+------+----------+------+------+----------+----------+
only showing top 3 rows

+----------+-----------+----------+
|first_name|  last_name|max_salary|
+----------+-----------+----------+
|    Yuping| Pargaonkar|     99999|
|      Mads|   Henseler|     99999|
|      Suvo|     Kakkar|     99998|
|   Odoardo|  Muchinsky|     99998|
|     Jinya|      Kroha|     99997|
|      Duro|     Zaian